In [167]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import scrapy as sc

from pathlib import Path
import os
import zipfile

import time
import re
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

from clickhouse_driver import Client

#Перманентно поменял настройки вывода графиков
from matplotlib import pylab
from pylab import *
pylab.rcParams['figure.figsize'] = (18.0, 6.0)
plt.rcParams.update({'font.size': 13})

#Скрыл вывод предупреждений.
import warnings
warnings.filterwarnings('ignore') #чтобы вернуть: (action='once')

# Глобально снял ограничение на кол-во отображаемых результатов для каждой ячейки ввода кода.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Включил возможность форматировать стили текста с помощью метода printmd()
from IPython.display import Markdown, display, HTML
def printmd(string):
    display(Markdown(string))
    
# Снял ограничение на вывод кол-ва столбцов и ширины колонки.
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_colwidth', -1)

# Установил формат вывода в таблице на 2 знака после запятой.
pd.options.display.float_format = '{:,.3f}'.format
    
# Добавил функцию вывода таблиц в одну строку, для экономии пространства и улучшения восприятия информации.
def display_side_by_side(dfs:list, captions:list):
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

In [18]:
options =  Options()
# options.set_headless(True)
driver = webdriver.Firefox(options=options)
# driver.set_window_size(550, 550)

#Функция get_page получает исходный код страницы по указанному url.
def get_page(url):
    global page
    driver.get(url)
    page = BeautifulSoup(driver.page_source)
    
get_page('https://s3.amazonaws.com/tripdata/index.html')

In [27]:
page = BeautifulSoup(driver.page_source.encode('utf-8'))

In [63]:
page

<html><head>
<!--

  Amazon S3 Bucket listing.


  Copyright (C) 2008 Francesco Pasqualini

      This program is free software: you can redistribute it and/or modify
      it under the terms of the GNU General Public License as published by
      the Free Software Foundation, either version 3 of the License, or
      (at your option) any later version.

      This program is distributed in the hope that it will be useful,
      but WITHOUT ANY WARRANTY; without even the implied warranty of
      MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
      GNU General Public License for more details.

      You should have received a copy of the GNU General Public License
      along with this program.  If not, see <http://www.gnu.org/licenses/>.

  -->
<!--

  Modified by Nolan Lawson!  (http://nolanlawson.com).  I'm keeping the spirit of the
  GPL alive by issuing this with the same license!

  -->
<title>Index of bucket "tripdata"</title>
<link href="//netdna.bootstrapcdn.com

In [84]:
class HTMLTableParser:

    def parse_url(self, url):
        return [(table['class'],self.parse_html_table(table))\
                for table in url.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                if column.find('a'):
                    df.iat[row_marker,column_marker] = column.find('a')['href']
                else:
                    df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df
    
hp = HTMLTableParser()
table = hp.parse_url(page)[0][1][:-1] # Grabbing the table from the tuple
table

,Name,Date Modified,Size,Type
0,https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip,"Apr 30th 2018, 10:18:55 pm",16.79 MB,ZIP file
1,https://s3.amazonaws.com/tripdata/201307-201402-citibike-tripdata.zip,"Jan 19th 2017, 07:23:25 am",178.26 MB,ZIP file
2,https://s3.amazonaws.com/tripdata/201307-citibike-tripdata.zip,"Jan 19th 2017, 07:23:27 am",27.07 MB,ZIP file
3,https://s3.amazonaws.com/tripdata/201308-citibike-tripdata.zip,"Jan 19th 2017, 07:23:27 am",32.09 MB,ZIP file
4,https://s3.amazonaws.com/tripdata/201309-citibike-tripdata.zip,"Jan 19th 2017, 07:23:27 am",33.16 MB,ZIP file
...,...,...,...,...
189,https://s3.amazonaws.com/tripdata/JC-202203-citibike-tripdata.csv.zip,"Apr 7th 2022, 02:11:46 am",1.48 MB,ZIP file
190,https://s3.amazonaws.com/tripdata/JC-202204-citibike-tripdata.csv.zip,"May 4th 2022, 01:38:33 am",1.85 MB,ZIP file
191,https://s3.amazonaws.com/tripdata/JC-202205-citibike-tripdata.csv.zip,"Jun 4th 2022, 11:19:12 am",2.34 MB,ZIP file
192,https://s3.amazonaws.com/tripdata/JC-202206-citibike-tripdata.csv.zip,"Jul 16th 2022, 12:33:12 am",3.05 MB,ZIP file


In [88]:
temp_row = table['Name']
temp_row

0      https://s3.amazonaws.com/tripdata/201306-citibike-tripdata.zip       
1      https://s3.amazonaws.com/tripdata/201307-201402-citibike-tripdata.zip
2      https://s3.amazonaws.com/tripdata/201307-citibike-tripdata.zip       
3      https://s3.amazonaws.com/tripdata/201308-citibike-tripdata.zip       
4      https://s3.amazonaws.com/tripdata/201309-citibike-tripdata.zip       
                                    ...                                     
189    https://s3.amazonaws.com/tripdata/JC-202203-citibike-tripdata.csv.zip
190    https://s3.amazonaws.com/tripdata/JC-202204-citibike-tripdata.csv.zip
191    https://s3.amazonaws.com/tripdata/JC-202205-citibike-tripdata.csv.zip
192    https://s3.amazonaws.com/tripdata/JC-202206-citibike-tripdata.csv.zip
193    https://s3.amazonaws.com/tripdata/JC-202207-citbike-tripdata.csv.zip 
Name: Name, Length: 194, dtype: object

In [90]:
# temp_row = []
# for elems in page.find_all('a', href=True):
#     temp_row.append(elems['href'])
# temp_row

In [91]:
# Методы сохранения файлов
def get_file_file(url): #делаем запрос на сам файл
    try:
        r = requests.get(url, stream=True)
        return r
    except:
        print('Нет коннекта при соединении с файлом по указанной ссылке.')
        pass


def get_file_name(url): #получаем имя файла
    try:
        name = url.split('/')[-1]  #потрошим ссылку через / и берём оттуда последние данные
        return name
    except:
        return url

def save_file(name, file_object): #сохраняет файл в корневой папке
    try:
        path = './files/'+name
        with open(path, 'bw') as f:
            for chunk in file_object.iter_content(None):
                f.write(chunk)
            print(str(name)+' загружен')
    except:
        pass
    time.sleep(5)

In [23]:
# def new_files(x):
#     new_file = []
#     try:
#         for file in x:
#             new_file.append(file['src'])
#     except:
#         pass

# df['pictures'].apply(lambda x: new_files(x))

# clean_files = [x for x in new_file if x not in list(files)]
# len(clean_files)

In [32]:
# folder path
dir_path = r'./files/'

# list file and directories
res = os.listdir(dir_path)
print(res)

['201306-citibike-tripdata.zip', '201307-201402-citibike-tripdata.zip']


In [36]:
for file in res:
        print(file+" : "+ str(Path(dir_path+file).stat().st_size))

201306-citibike-tripdata.zip : 16785103
201307-201402-citibike-tripdata.zip : 0


In [115]:
def zip_extractor(target_file):
    endswith = '.csv'
    try:
        path = Path(target_file)
        target_file_size = path.stat().st_size
        
        with zipfile.ZipFile(target_file) as z:
            for file in z.namelist():
                if file.endswith(endswith):
                    if (target_file_size) > 0:
                        print("Извлекаемый из архива файл существует: "+file)
                        print()
                    else:
                        z.extract(file,'./files/csv/')
                print(file+" файл разархивирован", endswith)
    except ValueError as ve:
        print(ve)

In [116]:
for img in temp_row[:-1]:
    filename = img.split('/')[-1]
    try:
        path = Path(dir_path+filename)
        target_file_size = path.stat().st_size
    except:
        print("Файл скачивается: "+img)
        save_file(get_file_name(img), get_file_file(img))
    else:
        if (target_file_size) > 0:
            print("Файл существует: "+filename)
        else:
            print("Файл скачан не полностью, продолжается скачивание: "+img)
            save_file(get_file_name(img), get_file_file(img))
    zip_extractor('.\\'+str(path))

Файл существует: 201306-citibike-tripdata.zip
Извлекаемый из архива файл существует: 201306-citibike-tripdata.csv

201306-citibike-tripdata.csv файл разархивирован .csv
__MACOSX/ файл разархивирован .csv
Извлекаемый из архива файл существует: __MACOSX/._201306-citibike-tripdata.csv

__MACOSX/._201306-citibike-tripdata.csv файл разархивирован .csv
Файл существует: 201307-201402-citibike-tripdata.zip
Извлекаемый из архива файл существует: 2014-02 - Citi Bike trip data.csv

2014-02 - Citi Bike trip data.csv файл разархивирован .csv
Извлекаемый из архива файл существует: 2014-01 - Citi Bike trip data.csv

2014-01 - Citi Bike trip data.csv файл разархивирован .csv
Извлекаемый из архива файл существует: 2013-12 - Citi Bike trip data.csv

2013-12 - Citi Bike trip data.csv файл разархивирован .csv
Извлекаемый из архива файл существует: 2013-11 - Citi Bike trip data.csv

2013-11 - Citi Bike trip data.csv файл разархивирован .csv
Извлекаемый из архива файл существует: 2013-10 - Citi Bike trip da

In [346]:
df = pd.read_csv('./files/csv/2013-07 - Citi Bike trip data.csv')

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577703 entries, 0 to 577702
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   tripduration             577703 non-null  int64  
 1   starttime                577703 non-null  object 
 2   stoptime                 577703 non-null  object 
 3   start station id         577703 non-null  int64  
 4   start station name       577703 non-null  object 
 5   start station latitude   577703 non-null  float64
 6   start station longitude  577703 non-null  float64
 7   end station id           559644 non-null  float64
 8   end station name         559644 non-null  object 
 9   end station latitude     559644 non-null  float64
 10  end station longitude    559644 non-null  float64
 11  bikeid                   577703 non-null  int64  
 12  usertype                 577703 non-null  object 
 13  birth year               337382 non-null  float64
 14  gend

In [348]:
df['stop_date'] = pd.to_datetime(df['stoptime'])
df['stop_date'] = df['stop_date'].astype('datetime64[D]')
df.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,stop_date
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753,-73.970,504,1 Ave & E 15 St,40.732,-73.982,16950,Customer,\N,0,2013-07-01
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.750,-74.003,459,W 20 St & 11 Ave,40.747,-74.008,19816,Customer,\N,0,2013-07-01
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730,-73.991,237,E 11 St & 2 Ave,40.730,-73.987,14548,Subscriber,1980,2,2013-07-01
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.719,-73.993,499,Broadway & W 60 St,40.769,-73.982,16063,Customer,\N,0,2013-07-01
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.735,-73.992,410,Suffolk St & Stanton St,40.721,-73.985,19213,Subscriber,1986,1,2013-07-01


In [349]:
trips_count_daily = df.groupby('stop_date')['bikeid'].count().to_frame().reset_index().rename(columns={'bikeid':'trips_count','stop_date':'date'})
trips_count_daily.head(10)

,date,trips_count
0,2013-07-01,16543
1,2013-07-02,22732
2,2013-07-03,21851
3,2013-07-04,22218
4,2013-07-05,21895
5,2013-07-06,20513
6,2013-07-07,20526
7,2013-07-08,21583
8,2013-07-09,26633
9,2013-07-10,25732


In [350]:
avg_duration_daily = df.groupby('stop_date')['tripduration'].mean().to_frame().reset_index().rename(columns={'tripduration':'avg_trip_dur','stop_date':'date'})
avg_duration_daily.head(10)

,date,avg_trip_dur
0,2013-07-01,908.297
1,2013-07-02,962.466
2,2013-07-03,952.451
3,2013-07-04,"1,264.269"
4,2013-07-05,"1,098.051"
5,2013-07-06,"1,175.185"
6,2013-07-07,"1,105.110"
7,2013-07-08,"1,024.492"
8,2013-07-09,928.177
9,2013-07-10,949.696


In [354]:
gender_df = df.groupby('stop_date')['bikeid'].count().to_frame()
for sex in df['gender'].unique():
    gender_df['gender_'+str(sex)] = df[df['gender'] == sex].groupby('stop_date')['gender'].count().to_frame()
gender_daily = gender_df.reset_index().rename(columns={'stop_date':'date'})[['date','gender_0','gender_1','gender_2']]
gender_daily['gender_0'] = gender_daily['gender_0'].fillna(0).astype(int)
gender_daily['gender_1'] = gender_daily['gender_1'].fillna(0).astype(int)
gender_daily['gender_2'] = gender_daily['gender_2'].fillna(0).astype(int)
gender_daily.head(5)

,date,gender_0,gender_1,gender_2
0,2013-07-01,2693,11044,2806
1,2013-07-02,4074,14637,4021
2,2013-07-03,4134,13964,3753
3,2013-07-04,10686,8411,3121
4,2013-07-05,8228,10426,3241


In [230]:
client = Client(host='localhost', settings={'use_numpy': True}).from_url('clickhouse://default:@localhost:19000/default') #логин пароль порт по умолчанию, дефолтная БД тоже по умолчанию

In [345]:
result = client.execute("SHOW TABLES")
print(result)

[('avg_duration_daily',), ('gender_daily',), ('trips_count_daily',)]


In [358]:
client.execute('INSERT INTO `default`.gender_daily VALUES', gender_daily.to_dict('records'),types_check=True)
client.execute('OPTIMIZE TABLE `default`.gender_daily')
client.query_dataframe('SELECT * FROM gender_daily')

35

[]

,date,gender_0,gender_1,gender_2
0,2013-06-01,3466,3833,1191
1,2013-06-02,10813,4014,1296
2,2013-06-03,2962,3671,877
3,2013-06-04,6870,6927,1782
4,2013-06-05,7089,6864,1785
...,...,...,...,...
60,2013-07-31,4954,24959,7339
61,2013-08-01,57,97,26
62,2013-08-02,0,1,0
63,2013-08-07,0,1,0


In [359]:
client.execute('INSERT INTO `default`.trips_count_daily VALUES', trips_count_daily.to_dict('records'),types_check=True)
client.execute('OPTIMIZE TABLE `default`.trips_count_daily')
client.query_dataframe('SELECT * FROM trips_count_daily')


35

[]

,date,trips_count
0,2013-06-01,8490
1,2013-06-02,16123
2,2013-06-03,7510
3,2013-06-04,15579
4,2013-06-05,15738
...,...,...
60,2013-07-31,37252
61,2013-08-01,180
62,2013-08-02,1
63,2013-08-07,1


In [360]:
# client.execute('''CREATE TABLE test (
#     `a` Int64,
#     ---`b` Nullable(Float64),
#     `c` Nullable(String)
#     ) Engine = MergeTree()
#     ORDER BY `a`;'''
# )



# df2 = pd.DataFrame({
#     'a': [1, 1, 1],
#     'c': ['a', 'sa', None],
# }, dtype=object)
# df2
client.execute('INSERT INTO `default`.avg_duration_daily VALUES', avg_duration_daily.to_dict('records'),types_check=True)
client.execute('OPTIMIZE TABLE `default`.avg_duration_daily')
client.query_dataframe('SELECT * FROM avg_duration_daily')

35

[]

,date,avg_trip_dur
0,2013-06-01,"1,423.124"
1,2013-06-02,"2,059.949"
2,2013-06-03,"1,667.470"
3,2013-06-04,"1,917.661"
4,2013-06-05,"2,330.615"
...,...,...
60,2013-07-31,921.892
61,2013-08-01,"6,286.739"
62,2013-08-02,"217,948.000"
63,2013-08-07,"730,955.000"
